### Placing parentheses

- Imagine you are given an arithmetic sequence. How can we place brackets such that we get the maximum value of such a sequence?
    - For example, given `5 - 8 + 7 X 4 - 8 + 9`, what is the largest value I can get from this by adding brackets?
        - $(5 - 8 + 7) * (4 - 8 + 9) = 4 * 5 = 20$
        - $(5 - (8 + 7)) * (4 - (8 + 9)) = -10 * -13 = 130$
    - **Input:** A sequence of digits $d_1 ... d_n$ and operations $op_1, ... op_{n-1} \in \{ +, - , \times \}$
    - **Output:** An order of applying the operations that maximises the expression 


- Let's first identify what the subproblems are to establish the recurrence
    - For any given expression, we can break it into 2 subexpressions on the left and right
        - For example, in `5 - 8 + 7 X 4 - 8 + 9`, we can split the expression into 5 possible expressions (at each of the operations)
            1. `5`, `-`, `8 + 7 X 4 - 8 + 9`
            2. `5 - 8`, `+`, `7 X 4 - 8 + 9`
            3. `5 - 8 + 7`, `X`, `4 - 8 + 9`
            4. `5 - 8 + 7 X 4`, `-`, `8 + 9`
            5. `5 - 8 + 7 X 4 - 8`, `+`, `9`
        - For each possible split, we need to test 4 possibilities. Why? Because depending on the operation, and the sign (operations can give negative values), we might want the min or max!
            - $\min(\text{left}) \{+, -, \times \} \min(\text{right})$
            - $\min(\text{left}) \{+, -, \times \} \max(\text{right})$
            - $\max(\text{left}) \{+, -, \times \} \min(\text{right})$
            - $\max(\text{left}) \{+, -, \times \} \max(\text{right})$
        - This becomes a recursive problem, where we can set up the same 4 possibilities for sub-subarrays 
    - The recursion continues until we reach the base case with 1 or 2 digits (no way to bracket, min and max are equal) 

    - For every subproblem, we want to record the possible minimum and maximum value

- So we build up 2 2-by-2 arrays, where the axis of each array will hold a `from` index and a `to` index
    - The first array will hold the minimum values of all subarrays, and the second will hold all maximum values

- Using this, we can compute the optimal value for each element

In [223]:
import re
import math 
input_string = "5-8+7*4-8+9"
digits = [int(x) for x in re.split("\-|\+|\*", input_string)]
ops = re.findall("\-|\+|\*", input_string)

def place_parentheses(input_string, return_arr = False):
    '''
    Time complexity: O(N^3), because there are 3 loops; 2 to iterate through the 2D array, and 1 to test every combination of values for a given substring. 
    Space complexity: O(N^2) because of the 2 2D arrays storing values
    '''
    digits = [int(x) for x in re.split("\-|\+|\*", input_string)]
    ops = re.findall("\-|\+|\*", input_string)

    min_arr = [['' for _ in range(len(digits))] for _ in range(len(digits))]
    max_arr = [['' for _ in range(len(digits))] for _ in range(len(digits))]

    for i in range(len(digits)):
        for j in range(len(digits)-i):
            ## Iterate through the 2D array diagonally. So it goes 0,0 | 1,1 | 2,2 ... THEN 0,1 | 1,2 | 2,3 ... THEN 0,2 | 1,3 ...
            minval = math.inf
            maxval = -math.inf
            if j == i+j:
                ## For a given (j, i+j) position denoting the start and end of a substring, if j == i+j, then there are no order of operations to consider. Return the value
                min_arr[j][i+j] = digits[j]
                max_arr[j][i+j] = digits[j]
                continue
            for leftarr_rightbound in range(j, i+j):
                ## For a given (j, i+j) position denoting the start and end of a substring, iterate through every possible *additional* bracket. So you only try 1 more possible bracket for each substring, because you have already computed the optimal value for every substring to that point
                leftval_min = min_arr[j][leftarr_rightbound] 
                leftval_max = max_arr[j][leftarr_rightbound]
                op = ops[leftarr_rightbound]
                rightval_min = min_arr[leftarr_rightbound+1][i+j]
                rightval_max = max_arr[leftarr_rightbound+1][i+j]

                ## Compare every combination min and max values on the left and right side of the split, and return the min and max values
                minmin = eval(str(leftval_min) + op + str(rightval_min))
                maxmax = eval(str(leftval_max) + op + str(rightval_max))
                minmax = eval(str(leftval_min) + op + str(rightval_max))
                maxmin = eval(str(leftval_max) + op + str(rightval_min))
            
                minval = min(minval, minmin, maxmax, minmax, maxmin)
                maxval = max(maxval, minmin, maxmax, minmax, maxmin)

                
            min_arr[j][i+j] = minval
            max_arr[j][i+j] = maxval
    
    if return_arr:
        return max(min_arr[0][len(digits)-1], max_arr[0][len(digits)-1]), min_arr, max_arr
    return max(min_arr[0][len(digits)-1], max_arr[0][len(digits)-1])

val, min_arr, max_arr = place_parentheses(input_string, return_arr=True)

- Naturally, we want to know how we got to the maximum value from the function above. Similar to previous problems, we can trace our way through the 2 arrays!

In [233]:
input_string = "5-8+7*4-8+9"
digits = [int(x) for x in re.split("\-|\+|\*", input_string)]
ops = re.findall("\-|\+|\*", input_string)

def trace_bracket_positions(min_arr, max_arr):
    '''
    Time complexity: O(N^2). O(N) comes from stepwise backward tracing from the maximum value, and O(N) from iterating through all possible midpoints.
    Space complexity: O(N) from the additional space needed to store brackets, queue, and set of elements used
    '''

    curr_val = max_arr[0][len(max_arr)-1]

    ## Init queue, starting from the top right
    elem_queue = [('max_arr', 0, len(digits)-1)]
    elems_used = set()
    bracket_positions = []

    # Iterate over all possible splits
    while len(elem_queue) != 0:
        # print('='*50)
        # print(elem_queue)
        arr, row, col = elem_queue.pop(0)
        curr_val = eval(arr)[row][col]

        for midpoint in range(row, col):
            leftarr_row, leftarr_col = (row, midpoint)
            rightarr_row, rightarr_col = (midpoint+1, col)
            op = ops[midpoint]
            # print(leftarr_row, leftarr_col, rightarr_row, rightarr_col)
            
            minmin = eval(str(min_arr[leftarr_row][leftarr_col]) + op + str(min_arr[rightarr_row][rightarr_col]))
            maxmax = eval(str(max_arr[leftarr_row][leftarr_col]) + op + str(max_arr[rightarr_row][rightarr_col]))
            minmax = eval(str(min_arr[leftarr_row][leftarr_col]) + op + str(max_arr[rightarr_row][rightarr_col]))
            maxmin = eval(str(max_arr[leftarr_row][leftarr_col]) + op + str(min_arr[rightarr_row][rightarr_col]))
            
            if curr_val in [minmin, maxmax, minmax, maxmin]:
                bracket_positions.append((leftarr_row, leftarr_col))
                bracket_positions.append((rightarr_row, rightarr_col))

                if (leftarr_col - leftarr_row) <= 1:
                    [elems_used.add(x) for x in range(leftarr_row, leftarr_col+1)]
                else:
                    if curr_val in [minmin, minmax]:
                        elem_queue.append(('min_arr', leftarr_row, leftarr_col))
                    elif curr_val in [maxmax, maxmin]:
                        elem_queue.append(('max_arr', leftarr_row, leftarr_col))

                if (rightarr_col - rightarr_row) <= 1:
                    [elems_used.add(x) for x in range(rightarr_row, rightarr_col+1)]
                else:
                    if curr_val in [minmin, maxmin]:
                        elem_queue.append(('min_arr', rightarr_row, rightarr_col))
                    elif curr_val in [maxmax, minmax]:
                        elem_queue.append(('max_arr', rightarr_row, rightarr_col))
                break

    return bracket_positions

def return_bracketed_expression(input_string, bracket_positions):
    for position in bracket_positions:
        start_element, end_element = position
        digits[start_element] = '(' + str(digits[start_element])
        digits[end_element] = str(digits[end_element]) + ')'
    final_expression = ''.join([x+y for x,y in zip(digits, ops+[''])])
    
    return final_expression

bracket_positions = trace_bracket_positions(min_arr, max_arr)
print(return_bracketed_expression(input_string, bracket_positions))
eval(return_bracketed_expression(input_string, bracket_positions))

(5)-((8+7)*((4)-(8+9)))


200